In [ ]:
import asyncio

In [ ]:
async def count():
    print('One')
    y = await asyncio.sleep(1)
    print('Two')
    return y
    
async def main():
    await asyncio.gather(count(), count(), count())
    #await count(),count()


In [ ]:
import time
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f' executed in {elapsed:0.2f} seconds.')

In [ ]:
def count():
    print('One')
    time.sleep(1)
    print('Two')

In [ ]:
def main ():
    for _ in range(3):
        count()

In [ ]:
s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f' executed in {elapsed:0.2f} seconds.')

In [ ]:
import random

In [ ]:
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

In [ ]:
async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f'Initiated makerandom({idx})')
    i = random.randint(0,10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

In [ ]:
random.seed(444)
r1, r2, r3 = await (main())
print()
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

## ASYNC LOGGER 

In [2]:
import json
import os
import pandas as pd
import numpy as np
import asyncio
from eth_account import account
from web3 import Web3
from src import s3_services
from src import sql_manager
from datetime import datetime
from importlib import reload

In [47]:
#Web3 Connection Handler
projectID = '69765a3368f44f5c8cc3691d2361e496'

infura_url = f'https://mainnet.infura.io/v3/{projectID}'
web3 = Web3(Web3.HTTPProvider(infura_url))

infura_url = f'wss://mainnet.infura.io/ws/v3/{projectID}'
web3ws = Web3(Web3.WebsocketProvider(infura_url))

if web3.isConnected() and web3ws.isConnected():
    print('The connection with Ethereum network is successfull')
else:
    print('Network connection error')

The connection with Ethereum network is successfull


In [ ]:
web3ws.eth.get_transaction_by_block()

In [4]:
# Ronin Bridge Smart Contract
address = '0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2'
abi = json.loads('[{"inputs":[{"internalType":"address","name":"_proxyTo","type":"address"},{"internalType":"address","name":"_registry","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_oldAdmin","type":"address"},{"indexed":true,"internalType":"address","name":"_newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_oldAdmin","type":"address"}],"name":"AdminRemoved","type":"event"},{"anonymous":false,"inputs":[],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_new","type":"address"},{"indexed":true,"internalType":"address","name":"_old","type":"address"}],"name":"ProxyUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"_depositId","type":"uint256"},{"indexed":true,"internalType":"address","name":"_owner","type":"address"},{"indexed":true,"internalType":"address","name":"_tokenAddress","type":"address"},{"indexed":false,"internalType":"address","name":"_sidechainAddress","type":"address"},{"indexed":false,"internalType":"uint32","name":"_standard","type":"uint32"},{"indexed":false,"internalType":"uint256","name":"_tokenNumber","type":"uint256"}],"name":"TokenDeposited","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"_withdrawId","type":"uint256"},{"indexed":true,"internalType":"address","name":"_owner","type":"address"},{"indexed":true,"internalType":"address","name":"_tokenAddress","type":"address"},{"indexed":false,"internalType":"uint256","name":"_tokenNumber","type":"uint256"}],"name":"TokenWithdrew","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpaused","type":"event"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"constant":true,"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_newAdmin","type":"address"}],"name":"changeAdmin","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"depositCount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"deposits","outputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"tokenAddress","type":"address"},{"internalType":"address","name":"sidechainAddress","type":"address"},{"internalType":"uint32","name":"standard","type":"uint32"},{"internalType":"uint256","name":"tokenNumber","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"pause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"proxyType","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"registry","outputs":[{"internalType":"contract Registry","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"removeAdmin","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"unpause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_newProxyTo","type":"address"}],"name":"updateProxyTo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_registry","type":"address"}],"name":"updateRegistry","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"withdrawals","outputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"tokenAddress","type":"address"},{"internalType":"uint256","name":"tokenNumber","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]')
contract = web3.eth.contract(address=address, abi = abi)

In [ ]:
contract.events.TokenWithdrew.event_name

In [24]:
block_data = pd.DataFrame()

In [ ]:
web3.eth.get_transaction()

In [66]:
def handle_event(event, df):
    #print(Web3.toJSON(event))
    #event_frame = pd.DataFrame.from_dict(event.args)
    print(event['args'])
    print(f'Movement found in {event.blockNumber}')
    print(f'address: {event.address}')
    print(f'TransactionHash: {event.transactionHash}')
    #print(web3.eth.get_block(event_blockNumber))
    print(web3.eth.get_transaction(web3.toHex(event.transactionHash)))
    #df = df.append(event_frame)
    #print(df)

In [49]:
async def log_loop(event_filter, poll_interval):
    for i in range(10):
        for TokenDeposited in event_filter_1.get_new_entries():
            print('New Deposit')
            handle_event(TokenDeposited, block_data)
        for TokenWithdrew in event_filter_2.get_new_entries():
            print('New Withdrawal')
            handle_event(TokenWithdrew, block_data)
        
        await asyncio.sleep(poll_interval)

In [64]:
event_filter_1 = contract.events.TokenDeposited.createFilter(fromBlock='latest')
event_filter_2 = contract.events.TokenWithdrew.createFilter(fromBlock='latest')
#block_filter = web3.eth.filter('latest')
#tx_filter = web3.eth.filter('pending')

async def main():
    loop = asyncio.get_event_loop()
    await (asyncio.gather(log_loop(event_filter,poll_interval=2)))

In [68]:
await main()

New Deposit
AttributeDict({'_depositId': 1795301, '_owner': '0x97b6EeE50fCcC36212FF74521c2a4F3b3E38f735', '_tokenAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '_sidechainAddress': '0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5', '_standard': 20, '_tokenNumber': 410000000000000000})
Movement found in 13451703
address: 0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2
TransactionHash: b'\xa6\x06\x96@G\x17\x9a\x89\xb1\x13hN*\xcc*\x1e\x16\xeb\x1d\xe8#o\xbfTAOvk\x12\xa3\x97\x11'
AttributeDict({'accessList': [], 'blockHash': HexBytes('0xc425011cd1c2cfeb8bcfb90376f5bd6404ccff4c039d52cea04fb10cba0269b7'), 'blockNumber': 13451703, 'chainId': '0x1', 'from': '0x46C16171932f651C5AEf4cC98A3D86D4eDD6f569', 'gas': 215887, 'gasPrice': 60044880662, 'hash': HexBytes('0xa606964047179a89b113684e2acc2a1e16eb1de8236fbf54414f766b12a39711'), 'input': '0xeee3f07a00000000000000000000000097b6eee50fccc36212ff74521c2a4f3b3e38f735', 'maxFeePerGas': 80147416820, 'maxPriorityFeePerGas': 1940000000, 'nonce': 3, 'r': H

AttributeDict({'accessList': [], 'blockHash': HexBytes('0x700baa5c36bdfd8846f9eb65411464b791fd0f47fe363815139475dfdd593616'), 'blockNumber': 13451709, 'chainId': '0x1', 'from': '0x6470Ad27331a355F7C2E3B8c1aAAcC84F026B422', 'gas': 215887, 'gasPrice': 53895864974, 'hash': HexBytes('0xe6980f22e81247a51bf4d72b9bb5e6289f7de7e32ddf397b60abf9365fbd9bd6'), 'input': '0xeee3f07a0000000000000000000000009c58a7a4efc3dc08a63a3ae349eb47ecca112cf8', 'maxFeePerGas': 56446339465, 'maxPriorityFeePerGas': 1500000000, 'nonce': 5, 'r': HexBytes('0xaf86a93e28a86efafcbeeee524609db3aa40608a086140010ddbfa6238514624'), 's': HexBytes('0x2e6062c8198565e181828ab2580b43702ca51ab8487f09632192747b3a4ec6c9'), 'to': '0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2', 'transactionIndex': 255, 'type': '0x2', 'v': 1, 'value': 110000000000000000})
New Deposit
AttributeDict({'_depositId': 1795309, '_owner': '0x49F9627dB8Ca38590b84cd76589da3CdF356ee62', '_tokenAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '_sidechainAddre

AttributeDict({'accessList': [], 'blockHash': HexBytes('0x1ad4d84a77dc83861dbdff961b7809da805c439c6a9ef5dae3161bf063d9b780'), 'blockNumber': 13451712, 'chainId': '0x1', 'from': '0x7c02209Fc1E335060E381c0c64d88Ef1CeFAd5aa', 'gas': 215887, 'gasPrice': 61738189009, 'hash': HexBytes('0x7e14e8761be1c95d10d8732e0962a17c31beba0e7f4a0e0225edbd4f90813c47'), 'input': '0xeee3f07a00000000000000000000000076610e99199a121290d229c01fc0bb39db3340b9', 'maxFeePerGas': 75899110935, 'maxPriorityFeePerGas': 1500000000, 'nonce': 0, 'r': HexBytes('0x5fa210bbcc2cd2adf470bacb55f0c9fa35d39b3acbe58088378932903809c3c8'), 's': HexBytes('0x7bb2edda66372264b97db07034b63a4f357a8476a94bfea79450b02abb018e39'), 'to': '0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2', 'transactionIndex': 116, 'type': '0x2', 'v': 1, 'value': 254000000000000000})
New Withdrawal
AttributeDict({'_withdrawId': 960923, '_owner': '0xA6c20acbC2960E7EDf225A6DD0C2A64Cc2C94A05', '_tokenAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '_tokenNumber

In [34]:
block_data

""


In [ ]:
import asyncio

async def rxer():
    i=0
    while True:
        i+=1
        print ('Rxer ',i)
        await asyncio.sleep(1)

async def WsServe():
    for i in range(5):
        print ('WsServe',i)
        await asyncio.sleep(1)

    print ('Finish')
    loop.stop()
    loop.close()

loop=asyncio.get_event_loop()
loop.create_task(rxer())
loop.run_until_complete(WsServe())
loop.run_forever()

In [ ]:
async def WsServe(stop_event):
    for i in range(5):
        print ('WsServe',i)
        await asyncio.sleep(1)

    print ('Finish')
    stop_event.set()
    await asyncio.sleep(1)

async def main():
    asyncio.get_event_loop().create_task(rxer())
    stop_event = asyncio.Event()
    asyncio.get_event_loop().create_task(WsServe(stop_event))
    await stop_event.wait()

asyncio.run(main())

# python 3.6 and older:
#asyncio.get_event_loop().run_until_complete(main())